In [1]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc

In [2]:
num_qubits = 2
max_weight = 1

HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    intersect = used_indices_1.intersection(used_indices_2)
    if len(intersect) > 0 and len(intersect) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
measure_string_attributes = list([p for p in measure_string_iterator])

hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        measure_string_attributes,
        measure_string_attributes,
    )
)



import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[1:] for s in el[-4:-1])
            egen = ",".join(str(s) for s in el[:-4])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])
ic(df)

ic| df:           H,+X_  H,+Y_  H,+Z_  H,+_X  H,+_Y  H,+_Z  S,+X_  S,+Y_  S,+Z_  \
        XX,XX,X_      0      0      0      0      0      0      0     -2     -2   
        XX,XX,_X      0      0      0      0      0      0      0      0      0   
        XX,XX,XX      0      0      0      0      0      0      0     -2     -2   
        XX,XY,X_      0      0      0      0      0      0      0     -2     -2   
        XX,XY,_Y      0      0      0      0      0      2      0      0      0   
        ...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
        ZZ,ZY,_Y      0      0      0     -2      0      0      0      0      0   
        ZZ,ZY,ZY      0      0      0     -2      0      0      0      0      0   
        ZZ,ZZ,Z_      0      0      0      0      0      0     -2     -2      0   
        ZZ,ZZ,_Z      0      0      0      0      0      0      0      0      0   
        ZZ,ZZ,ZZ      0      0      0      0      0      0     -2     -2      0   
    

,"H,+X_","H,+Y_","H,+Z_","H,+_X","H,+_Y","H,+_Z","S,+X_","S,+Y_","S,+Z_","S,+_X",...,"C,+Y_,+Z_","C,+_X,+_Y","C,+_X,+_Z","C,+_Y,+_Z","A,+X_,+Y_","A,+X_,+Z_","A,+Y_,+Z_","A,+_X,+_Y","A,+_X,+_Z","A,+_Y,+_Z"
"XX,XX,X_",0,0,0,0,0,0,0,-2,-2,0,...,0,0,0,0,0,0,-4,0,0,0
"XX,XX,_X",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-4
"XX,XX,XX",0,0,0,0,0,0,0,-2,-2,0,...,0,0,0,0,0,0,-4,0,0,-4
"XX,XY,X_",0,0,0,0,0,0,0,-2,-2,0,...,0,0,0,0,0,0,-4,0,0,0
"XX,XY,_Y",0,0,0,0,0,2,0,0,0,0,...,0,2,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"ZZ,ZY,_Y",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,4,0
"ZZ,ZY,ZY",0,0,0,-2,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,4,0
"ZZ,ZZ,Z_",0,0,0,0,0,0,-2,-2,0,0,...,0,0,0,0,-4,0,0,0,0,0
"ZZ,ZZ,_Z",0,0,0,0,0,0,0,0,0,-2,...,0,0,0,0,0,0,0,-4,0,0


In [3]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [9]:
import pygsti
from pygsti.extras import idletomography as idt
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2, 4, 8]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): 1}
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1
    )
from pygsti.baseobjs import Label

updated_ckt_list = []
for ckt in idle_experiments:
    new_ckt = ckt.copy(editable=True)
    for i, lbl in enumerate(ckt):
        # print(i, lbl)
        if lbl == Label(()):
            new_ckt[i] = [Label(("Gi", i)) for i in range(num_qubits)]
            # new_ckt[i] = Label(('Gi', ))
    # print(new_ckt)
    updated_ckt_list.append(new_ckt)
    mdl_datagen = pygsti.models.create_crosstalk_free_model(
    pspec, lindblad_error_coeffs={"Gi": {"SX": 0.01}}
)
# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 100000, seed=8675309, sample_error="none"
)
ds

In [14]:
from idttools import allerrors, all_full_length_observables
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=((),),):
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    print(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_observables = all_full_length_observables(
            pauli_fidpair[1], nqubits
        )
        infos_for_this_fidpair = _collections.OrderedDict()
        for j, out in enumerate(all_observables):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            #info["jacobian row"] = full_jacobian[ifp]
            infos_for_this_fidpair[out] = info
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[pauli_fidpair] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            # obs_err_rates = np.concatenate([np.array([
            #                 observed_error_rates[i]
            #                 for i in range(len(all_fidpairs))
            #                 ]
            #             )
            #         ]
            #     )
    return observed_error_rates

In [15]:
observed_error_rates = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

In [16]:
observed_error_rates

{0: [6.852828920911803e-18],
 1: [6.3227824255950655e-18],
 2: [6.3227824255950655e-18],
 3: [6.852828920911803e-18],
 4: [0.0],
 5: [0.0],
 6: [0.0],
 7: [0.0],
 8: [0.0],
 9: [0.0],
 10: [0.0],
 11: [0.0],
 12: [0.0],
 13: [0.0],
 14: [0.0],
 15: [0.0],
 16: [0.0],
 17: [0.0],
 18: [0.0],
 19: [0.0],
 20: [0.0],
 21: [0.0],
 22: [0.0],
 23: [0.0],
 24: [0.0],
 25: [0.0],
 26: [0.0],
 27: [0.0],
 28: [0.0],
 29: [0.0],
 30: [0.0],
 31: [0.0],
 32: [0.0],
 33: [0.0],
 34: [0.0],
 35: [0.0],
 36: [0.0],
 37: [0.0],
 38: [0.0],
 39: [0.0],
 40: [6.852828920911803e-18],
 41: [6.3227824255950655e-18],
 42: [6.3227824255950655e-18],
 43: [6.852828920911803e-18],
 44: [0.0],
 45: [0.0],
 46: [0.0],
 47: [0.0],
 48: [0.0],
 49: [0.0],
 50: [0.0],
 51: [0.0],
 52: [0.0],
 53: [0.0],
 54: [0.0],
 55: [0.0],
 56: [0.0],
 57: [0.0],
 58: [0.0],
 59: [0.0],
 60: [0.0],
 61: [0.0],
 62: [0.0],
 63: [0.0],
 64: [0.0],
 65: [0.0],
 66: [0.0],
 67: [0.0],
 68: [0.0],
 69: [0.0],
 70: [0.0],
 71: [0.0]